# Common Code

This code block includes import statements, function definitions, and declarations of variables that are common to the rest of the script. It needs to be run once before the other code blocks.

**Note: some code in this block is found in the stand-alone file vb_common_code.py**

In [ ]:
import requests   # best library to manage HTTP transactions
from bs4 import BeautifulSoup # web-scraping library
import json
from time import sleep
import csv
import math
from fuzzywuzzy import fuzz # fuzzy logic matching
from fuzzywuzzy import process
import xml.etree.ElementTree as et # library to traverse XML tree
import urllib
import datetime
import string

# For a particular processing round, set a short name for the department here.
# This name is used to generate a set of unique processing files for that department.
testEmployer = 'Vanderbilt University' # to test against Wikidata employer property
employerQId = 'Q29052' # Vanderbilt University
deathDateLimit = '2000' # any death dates before this date will be assumed to not be a match
birthDateLimit = '1920' # any birth dates before this date will be assumed to not be a match
wikibase_instance_namespace = 'http://www.wikidata.org/entity/'

# ---------------------
# function definitions
# ---------------------

wikidata_endpoint_url = 'https://query.wikidata.org/sparql'
accept_media_type = 'application/json'
user_agent_header = 'VanderBot/1.7 (https://github.com/HeardLibrary/linked-data/tree/master/vanderbot; mailto:steve.baskauf@vanderbilt.edu)'

# NCBI identification requirements:
# tool name and email address should be sent with all requests
# see https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch
emailAddress = 'steve.baskauf@vanderbilt.edu' # put your email address here
toolName = 'VanderBot' # give your application a name here

# Extracts the local name part of an IRI, e.g. a qNumber from a Wikidata IRI
def extract_local_name(iri):
    # pattern is http://www.wikidata.org/entity/Q6386232
    pieces = iri.split('/')
    last_piece = len(pieces)
    return pieces[last_piece - 1]

# Generate the current UTC xsd:date
def generate_utc_date():
    whole_time_string_z = datetime.datetime.utcnow().isoformat() # form: 2019-12-05T15:35:04.959311
    date_z = whole_time_string_z.split('T')[0] # form 2019-12-05
    return date_z

today = generate_utc_date()

# generates a dictionary to be passed in a requests GET method to generate the request header
def generate_header_dictionary_non_sparql(acceptMediaType):
    userAgentHeader = 'VanderBot/1.3 (https://github.com/HeardLibrary/linked-data/tree/master/vanderbot; mailto:steve.baskauf@vanderbilt.edu)'
    requestHeaderDictionary = {
        'Accept' : acceptMediaType,
        'User-Agent': userAgentHeader
    }
    return requestHeaderDictionary

# write a list of lists to a CSV file
def write_lists_to_csv(fileName, array):
    with open(fileName, 'w', newline='', encoding='utf-8') as fileObject:
        writerObject = csv.writer(fileObject)
        for row in array:
            writerObject.writerow(row)

# Read from a CSV file into a list of dictionaries
def read_dicts_from_csv(filename):
    with open(filename, 'r', newline='', encoding='utf-8') as file_object:
        dict_object = csv.DictReader(file_object)
        array = []
        for row in dict_object:
            array.append(row)
    return array

# Write list of dictionaries to a CSV file
def write_dicts_to_csv(table, filename, fieldnames):
    with open(filename, 'w', newline='', encoding='utf-8') as csv_file_object:
        writer = csv.DictWriter(csv_file_object, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)

def lookup_department(researcher_qid, orcid_department, departments, department_labels):
    for department in departments:
        if researcher_qid == department['qid']:
            result = ''
            #print(department['affiliation'])
            for label in department_labels:
                if department['affiliation'] == label['qid']:
                    partial_ratio = fuzz.partial_ratio(orcid_department, label['label_en'])
                    #set_ratio = fuzz.token_set_ratio(orcid_department, label['label_en'])
                    if partial_ratio > 80:
                        #print('partial ratio', partial_ratio)
                        #print('set_ratio', set_ratio)
                        result = str(partial_ratio) + ' ' + orcid_department + '/' + label['label_en']
            return result

def find_surname_givens(name):
    # Get rid of periods and commas
    name = name.replace('.', ' ')
    name = name.replace(',', ' ')
    
    # Split name
    pieces = name.split(' ')
    
    # Get rid of empty pieces formed from extra spaces
    while '' in pieces:
        pieces.remove('')
    
    # Must be at least a surname and something else
    if len(pieces) <= 1:
        return False
    
    # Make sure first character is alphabetic
    # only fixes the case where there is one alphanumeric, but more than one is rare
    # typical cases are like (Kit) or "Kit"    
    for piece_index in range(len(pieces)):
        if not pieces[piece_index][0:1].isalpha(): 
            pieces[piece_index] = pieces[piece_index][1:len(pieces)] # remove the first non-alphabetic character
    # Now get rid of any empty strings; could also be caused by double spaces
    for piece in pieces:
        if len(piece) == 0: # there's nothing left, get rid of piece
            pieces.remove('')
            
    # Get rid of ", Jr.", "III", etc.
    if 'Jr' in pieces:
        pieces.remove('Jr')
    if 'Sr' in pieces:
        pieces.remove('Sr')
    if 'II' in pieces:
        pieces.remove('II')
    if 'III' in pieces:
        pieces.remove('III')
    if 'IV' in pieces:
        pieces.remove('IV')
    if 'V' in pieces:
        pieces.remove('V')
    
    # Not interested unless there are at least two pieces
    if len(pieces) == 1:
        return False
    
    # Put all but last piece together again
    given_names = ''
    for piece in pieces[0:len(pieces)-2]:
        given_names += piece + ' '
    given_names += pieces[len(pieces)-2]
    
    return {'given': given_names, 'family': pieces[len(pieces)-1]}

def generate_header_dictionary(accept_media_type,user_agent_header):
    request_header_dictionary = {
        'Accept' : accept_media_type,
#        'Content-Type': 'application/sparql-query',
        'Content-Type': 'application/x-www-form-urlencoded',
        'User-Agent': user_agent_header
    }
    return request_header_dictionary

# The following function requires the request header generated above
sparql_request_header = generate_header_dictionary(accept_media_type,user_agent_header)
# The query is a valid SPARQL query string

# Sends a query to the query service endpoint. 
# NOTE: request_header and endpoint are global variables defined earlier in the script
def send_sparql_query(query_string):
    # You can delete the two print statements if the queries are short. However, for large/long queries,
    # it's good to let the user know what's going on.
    print('querying SPARQL endpoint to acquire item metadata')
    #response = requests.post(endpoint, data=query_string.encode('utf-8'), headers=sparql_request_header)
    response = requests.post(wikidata_endpoint_url, data=dict(query=query_string), headers=sparql_request_header) # use URL-encoded method
    #print(response.text) # uncomment to view the raw response, e.g. if you are getting an error
    data = response.json()

    # Extract the values from the response JSON
    results = data['results']['bindings']
    
    print('done retrieving data')
    # print(json.dumps(results, indent=2))
    return results


# Manage altLabels

This process starts with altLabels downloaded from Wikidata and adds alternate names from ORCID

## Download Vanderbilt people's altLabels from Wikidata

Developed at https://github.com/HeardLibrary/linked-data/blob/master/publications/wikidata/download-vanderbilt-people-altlabels.py



In [ ]:
query = '''select distinct  ?person ?altLabel where {
  ?person p:P108 ?statement.
  {?statement ps:P108 wd:Q29052.}
  union
  {?statement ps:P108 wd:Q7914455.}
  ?person skos:altLabel ?altLabel.
  FILTER(lang(?altLabel)="en")
}'''

# The endpoint defaults to returning XML, so the Accept: header is required
r = requests.get(wikidata_endpoint_url, params={'query' : query}, headers={'Accept' : 'application/json'})

data = r.json()
#print(json.dumps(data,indent = 2))

table = [['qid', 'altLabel', 'source']]
items = data['results']['bindings']
for item in items:
    wikidata_iri = extract_local_name(item['person']['value'])
    alt_label = ''
    if 'altLabel' in item:
        alt_label = item['altLabel']['value']
    table.append([wikidata_iri, alt_label, 'wikidata'])
    
filename = 'vanderbilt_wikidata_altlabels.csv'
write_lists_to_csv(filename, table)
print('done')

In [ ]:
filename = 'vanderbilt_wikidata_altlabels.csv'
researcher_altlabels = read_dicts_from_csv(filename)
fieldnames = researcher_altlabels[0].keys() # get the field names from the existing file

filename = 'orcid_other_names.csv'
orcid_other = read_dicts_from_csv(filename)

filename = 'researchers.csv'
researcher_items = read_dicts_from_csv(filename)


In [ ]:
print(orcid_other[0])
print()
print(researcher_items[0])
print()
print(researcher_altlabels[0])

In [ ]:
for name in orcid_other[:100]:
    print(name['altName'], name['orcid'])
    for researcher in researcher_items:
        if name['orcid'] == researcher['orcid']:
            print(researcher['label_en'], researcher['qid'])
            break
    print()

# Query ORCID for Vanderbilt University people

Script developed at https://github.com/HeardLibrary/linked-data/blob/master/publications/orcid/orcid-get-json.ipynb

Retrieves results 100 at a time, then processes them by extracting desired information.  **NOTE: takes hours to run.**

Saves results in a file and the alternative names in a second file.

In [ ]:
table = [['orcid', 'given_names', 'family_name', 'start_date', 'end_date', 'department', 'organization']]
other_name_list = [['orcid', 'altName']]

# use the API to search for people associated with Vanderbilt University
# First search is for only one record, just to get the number of hits found
search_uri = 'https://pub.orcid.org/v2.0/search/?q=affiliation-org-name:"Vanderbilt+University"&start=1&rows=1'
accept_media_type = 'application/json'
response = requests.get(search_uri, headers = generate_header_dictionary_non_sparql(accept_media_type))
data = response.json()
#print(data)
number_results = data["num-found"]
print(data["num-found"])
number_pages = math.floor(number_results/100)
#print(number_pages)
remainder = number_results - 100*number_pages
#print(remainder)

for page_count in range(0, number_pages + 1):  # the remainder will be caught when page_count = number_pages
    print('page: ', page_count)
    search_uri = 'https://pub.orcid.org/v2.0/search/?q=affiliation-org-name:"Vanderbilt+University"&start='+str(page_count*100+1)
    response = requests.get(search_uri, headers={'Accept' : 'application/json'})
    print(response.url)
    data = response.json()
    orcids_dicts_list = data['result']

    # extract the identifier strings from the data structure
    orcids = []
    for orcid_dict in orcids_dicts_list:
        dictionary = {'id': orcid_dict['orcid-identifier']['path'], 'iri': orcid_dict['orcid-identifier']['uri']}
        orcids.append(dictionary)

    for orcid_index in range(0, len(orcids)):
        response = requests.get(orcids[orcid_index]['iri'], headers={'Accept' : 'application/json'})
        data = response.json()
        #print(json.dumps(data, indent = 2))
        orcid_id = data['orcid-identifier']['path']
        #print(orcid_id)
        # if there isn't a name, then go on to the next ORCID
        if not data['person']['name']:
            continue
        if data['person']['name']['given-names']:  
            given_names = data['person']['name']['given-names']['value']
        else:
            continue
        if data['person']['name']['family-name']:
            family_name = data['person']['name']['family-name']['value']
        # This has been a big pain when people don't have surnames.
        # It causes matches with everyone who has the same first name!
        else:
            continue
        #print(given_names, ' ', family_name)
        other_names = data['person']['other-names']['other-name']
        for other_name in other_names:
            #print(other_name['content'])
            other_name_list.append([orcid_id, other_name['content']])

        affiliations = data['activities-summary']['employments']['affiliation-group']
        #print(json.dumps(affiliations, indent = 2))
        for affiliation in affiliations:
            summaries = affiliation['summaries']
            #print(summaries)
            #print()
            for summary in summaries:
                employment = summary['employment-summary']
                #print(json.dumps(employment, indent = 2))
                start_date = ''
                if employment['start-date']:
                    if employment['start-date']['year']:
                        start_date += employment['start-date']['year']['value']
                        start_month = employment['start-date']['month']
                        if start_month:
                            start_date += '-' + start_month['value']
                            start_day = employment['start-date']['day']
                            if start_day:
                                start_date += '-' + start_day['value']
                #print('start date: ', start_date)
                end_date = ''
                if employment['end-date']:
                    if employment['end-date']['year']:
                        end_date += employment['end-date']['year']['value']
                        end_month = employment['end-date']['month']
                        if end_month:
                            end_date += '-' + end_month['value']
                            end_day = employment['end-date']['day']
                            if end_day:
                                end_date += '-' + end_day['value']
                #print('end date: ', end_date)
                department = employment['department-name']
                # if there is no value for department, set it to empty string
                if not department:
                    department = ''
                #print(department)
                if employment['organization']:
                    organization = employment['organization']['name']
                #print(organization)
                if 'Vanderbilt University' in organization:
                    print(orcid_id, given_names, family_name, start_date, end_date, department, organization)
                    table.append([orcid_id, given_names, family_name, start_date, end_date, department, organization])
                #print(table)
        sleep(.25)

print()
print('Done')
file_name = 'orcid_data.csv'
write_lists_to_csv(file_name, table)
file_name = 'orcid_other_names.csv'
write_lists_to_csv(file_name, other_name_list)


# Load data for upcoming cells

In [ ]:
filename = 'wikidata/researchers.csv'
researcher_items = read_dicts_from_csv(filename)
fieldnames = researcher_items[0].keys() # get the field names from the existing file

filename = 'vanderbilt_wikidata_altlabels.csv'
researcher_altlabels = read_dicts_from_csv(filename)

filename = 'orcid_other_names.csv'
orcid_other = read_dicts_from_csv(filename)

filename = 'orcid_data.csv'
orcid_people = read_dicts_from_csv(filename)

filename = 'wikidata/departments.csv'
departments = read_dicts_from_csv(filename)

filename = 'wikidata/department_labels.csv'
department_labels = read_dicts_from_csv(filename)

filename = 'wikidata/qid_not_equal_orcid.csv'
not_equal = read_dicts_from_csv(filename)

print(fieldnames)
print()
print(researcher_items[0])
print()
print(orcid_people[0])


# Check Wikidata people for new ORCIDs

Check the downloaded records for Wikidata against the ORCID download to see if any people in Wikidata whose employer is Vanderbilt have new ORCIDs.

In [ ]:
count = 1
#for researcher_item_index in range(500):
for researcher_item_index in range(len(researcher_items)):
    count +=1
    if count%100 == 0:
        print(count)
    # First screen is to only check items if their ORCID is unknown
    if researcher_items[researcher_item_index]['orcid'] == '':
        researcher_names = find_surname_givens(researcher_items[researcher_item_index]['label_en'])
        for orcid_person in orcid_people:
            bad_match = False
            # Second screen requires the surname of the checked ORCID to match the Wikidata label surname exactly
            # This prevents a high fraction of fuzzy matches where the last names are similar but not the same
            if researcher_names['family'] == orcid_person['familyName']: # require exact match to family name
                w_ratio = fuzz.WRatio(orcid_person['givenNames'] + ' ' + orcid_person['familyName'], researcher_items[researcher_item_index]['label_en'])

                # Third screen requires a high degree of similarity between the overall ORCID names and Wikidata labels
                if w_ratio > 90:
                    # The fourth screen eliminates near matches that have previously been determined by a human to be wrong
                    for mismatch in not_equal:
                        if mismatch['qid'] == researcher_items[researcher_item_index]['qid'] and mismatch['orcid'] == orcid_person['orcid']:
                            bad_match = True
                            # quit the mismatch list checking
                            break
                    if not bad_match:
                        #ratio = fuzz.ratio(orcid_person['givenNames'] + ' ' + orcid_person['familyName'], researcher_item['label_en'])
                        #partial_ratio = fuzz.partial_ratio(orcid_person['givenNames'] + ' ' + orcid_person['familyName'], researcher_item['label_en'])
                        #sort_ratio = fuzz.token_sort_ratio(orcid_person['givenNames'] + ' ' + orcid_person['familyName'], researcher_item['label_en'])
                        #set_ratio = fuzz.token_set_ratio(orcid_person['givenNames'] + ' ' + orcid_person['familyName'], researcher_item['label_en'])
                        print(w_ratio, researcher_items[researcher_item_index]['label_en'], wikibase_instance_namespace + researcher_items[researcher_item_index]['qid'])
                        dept_match = lookup_department(researcher_items[researcher_item_index]['qid'], orcid_person['department'], departments, department_labels)
                        print('   ', orcid_person['givenNames'] + ' ' + orcid_person['familyName'], 'https://orcid.org/' + orcid_person['orcid'])
                        print(orcid_person['department'])
                        if dept_match != '':
                            print('Dept. match:', dept_match)
                        print()
                        #print('name similarity ratio', ratio)
                        #print('partial ratio', partial_ratio)
                        #print('sort_ratio', sort_ratio)
                        #print('set_ratio', set_ratio)
                        #print('w_ratio', w_ratio)
                        researcher_items[researcher_item_index]['orcid'] = orcid_person['orcid']
                        researcher_items[researcher_item_index]['orcid_ref1_retrieved_val'] = today
                        write_dicts_to_csv(researcher_items, 'wikidata/researchers.csv', fieldnames)
                        break
                        
                    else:
                        # when a bad match was detected, go on to the next ORCID person
                        #print('detected:', researcher_items[researcher_item_index]['qid'], orcid_person['orcid'])
                        pass

print('done')

# Check new orcid download for people not yet associated with VU in Wikidata

This part of the script looks for people who ORCID said worked at Vanderbilt, but who don't have any employer property in Wikidata saying that they work(ed) at VU.

In [ ]:
print(len(orcid_people))
orcid_list = ''
for person in orcid_people:
    orcid_list += '"' + person['orcid'] + '"\n'
#print(orcid_list)

query_string = '''select distinct ?person ?name ?orcid where {
  values ?orcid
  {
''' + orcid_list +'''  }
  ?person wdt:P496 ?orcid.
  ?person rdfs:label ?name.
  filter(lang(?name) = "en")
  minus {
    {?person wdt:P108 wd:Q29052.}
    union
    {?person wdt:P108 wd:Q7914455.}
  }
  }'''

#print(query_string)

data = send_sparql_query(query_string)
#print(json.dumps(data, indent = 2))

missing_persons_list = []
#for person in data[:2]:
for person in data:
    person_dict = {}
    person_dict['name'] = person['name']['value']
    person_dict['orcid_url'] = 'https://orcid.org/' + person['orcid']['value']
    person_dict['orcid'] = person['orcid']['value']
    person_dict['qid'] = extract_local_name(person['person']['value'])
    person_dict['item_url'] = person['person']['value']
    missing_persons_list.append(person_dict)
#print(missing_persons_list)

fieldnames = ['name', 'orcid_url', 'orcid', 'item_url', 'qid']
filename = 'missing_persons.csv'
write_dicts_to_csv(missing_persons_list, filename, fieldnames)

# Issues to deal with:
# new people (this is good)
# LargeDatasetBot creates records in Wikidata with only initial when ORCID has full names.

print('done')

# Check for duplicate Wikidata items with the same ORCID


Also periodically run

```
select distinct ?person1 ?person2 ?orcid where {
  {?person1 wdt:P108 wd:Q29052.}
  union
  {?person1 wdt:P108 wd:Q7914455.}
  ?person1 wdt:P496 ?orcid.
  ?person2 wdt:P496 ?orcid.
  filter(?person1 != ?person2)
  }
order by ?orcid
```

In [ ]:
orcid_list = ''
for person in missing_persons_list:
    orcid_list += '"' + person['orcid'] + '"\n'
#print(orcid_list)

query_string = '''select distinct ?person1 ?person2 ?orcid where {
  values ?orcid
  {
''' + orcid_list +'''  }
  ?person1 wdt:P496 ?orcid.
  ?person2 wdt:P496 ?orcid.
  filter(?person1 != ?person2)
  }
order by ?orcid'''

print(query_string)
# This query can by copied and pasted into the Query Service
# Use those links to merge the duplicates.
# Then go back and rerun the code in the cell above.



# Run acquire_wikidata_metadata.py

Use the Q IDs saved in the `missing_persons.csv` file to create the spreadsheet for the new people to be added. Used a modified `config.json` (`config_minimal.json`) file to retrieve without the employer field, then add the employer column headers to create `researchers.csv`.

In [ ]:
filename = 'wikidata/missing_persons/researchers.csv'
missing_persons = read_dicts_from_csv(filename)
fieldnames = list(missing_persons[0].keys()) # get the field names from the existing file
print(missing_persons[0])

print()

filename = 'orcid_data.csv'
orcid_people = read_dicts_from_csv(filename)
print(orcid_people[0])

In [ ]:
#for person_index in range(50):
for person_index in range(len(missing_persons)):
    found = False
    for orcid_person in orcid_people:
        if missing_persons[person_index]['orcid'] == orcid_person['orcid']:
            found = True
            break
        found = False
    if not found:
        print('did not find')
    else:
        # Note: these values seem to be controlled, so fuzzy matching isn't required
        if 'Vanderbilt University Medical Center' == orcid_person['organization']:
            missing_persons[person_index]['employer'] = 'Q7914455'
        # Checked manually and all others are VU or some unit of VU
        else:
            missing_persons[person_index]['employer'] = 'Q29052'
        if orcid_person['startDate'] != '':
            missing_persons[person_index]['employer_start_time_val'] = orcid_person['startDate']
        if orcid_person['endDate'] != '':
            missing_persons[person_index]['employer_end_time_val'] = orcid_person['endDate']
        missing_persons[person_index]['employer_ref1_referenceUrl'] = 'https://orcid.org/' + orcid_person['orcid']
        # Manually add the date when the ORCID download was made
        missing_persons[person_index]['employer_ref1_retrieved_val'] = '2021-03-27'
        department = orcid_person['department']
        if 'Department of ' in department:
            department = department.split('Department of ')[1]
        missing_persons[person_index]['department'] = department
        missing_persons[person_index]['organization'] = orcid_person['organization']

filename = 'wikidata/missing_persons/researchers.csv'
write_dicts_to_csv(missing_persons, filename, fieldnames + ['department', 'organization'])

print('done')

After running this cell, I went through and googled people to determine their gender manually.

Modified the `config.json` to include department within same CSV, then generated the `csv-metadata.json` and CSV header.

Sorted by department, then pasted in the appropriate department Q ID into the cell.

After the upload, copied the new data into the `researchers.csv` and `departments.csv` files in the parent directory. I then reran the download script to clean up the data and remove the duplicates.